In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import json
from urllib.request import urlopen
import requests

from keys.keys import *

keys = Keys()

In [2]:
#stuff_to_build: Get news on days with strong moves like 3%+, get discounted cash flow, analyze future, analyze manager perfomance like payment in correlation to stock perfomance, get earnings history, compare to industry and peers
#stuff_to_build portfolio analysis: perfomance of portfolio, custom etf data from companies in index, compare perfomance of holdings to sector/ industry, correlations, risk etc of holdings

In [22]:
#data["financialData"]["financialData"]["currentPrice"]["raw"]
#data["incomeStatementHistory"]["incomeStatementHistory"]["incomeStatementHistory"][0]["interestExpense"]["raw"]
#data["balanceSheetHistory"]["balanceSheetHistory"]["balanceSheetStatements"][0]["totalAssets"]["raw"]
#data["assetProfile"]["assetProfile"]["sector"]
data["incomeStatementHistory"]["incomeStatementHistory"]["incomeStatementHistory"][0]["netIncomeApplicableToCommonShares"]["raw"]

-976091000

In [2]:
class Analysis:
    def __init__(self, security):
        self.name = security
        self.data_categories = ["assetProfile", "financialData", "defaultKeyStatistics", "calendarEvents" , "incomeStatementHistory", "cashflowStatementHistory", "balanceSheetHistory"]
        self.data = {"data": "value"}
        self.data_frame = {"data": "value"}
        
        for categorie in self.data_categories:
            self.api_Data = urlopen(f"https://query2.finance.yahoo.com/v10/finance/quoteSummary/{self.name}?modules={categorie}")
            self.json_data = self.api_Data.read().decode('utf-8', 'replace')
            self.json_data =  json.loads(self.json_data)
            self.data[categorie] = pd.DataFrame.from_records(self.json_data["quoteSummary"]["result"][0])
            
        self.full_name = self.data["assetProfile"]["assetProfile"]["sector"]
        self.website = self.data["assetProfile"]["assetProfile"]["website"]
        self.sector = self.data["assetProfile"]["assetProfile"]["sector"]
        self.industry = self.data["assetProfile"]["assetProfile"]["industry"]
        self.employees = self.data["assetProfile"]["assetProfile"]["fullTimeEmployees"]
        self.country = self.data["assetProfile"]["assetProfile"]["country"]
              
        self.api_data_prices = urlopen(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={self.name}&apikey={keys.alpha_key}")
        self.json_data_prices = self.api_data_prices.read().decode('utf-8', 'replace')
        self.json_data_prices =  json.loads(self.json_data_prices)
        self.prices = pd.DataFrame.from_records(self.json_data_prices)
        self.prices = self.prices.drop(axis=1, labels="Meta Data")
        self.prices = self.prices.dropna(axis=0)
        
    def get_full_analysis(self):
        self.get_present_value()
        self.get_intrinsic_pricing_discounted_cash_flow()
        self.get_intrinsic_discount()
        self.get_earnings_perfomance()
            
    def get_present_value(self):
        # discounted cash flow
        self.fcf = self.data["financialData"]["financialData"]["freeCashflow"]["raw"]
        self.equity_beta = self.data["defaultKeyStatistics"]["defaultKeyStatistics"]["beta"]["raw"]
        #debt_percentage = data["balanceSheetHistory"]["balanceSheetHistory"]["balanceSheetStatements"][0]["totalLiab"]["raw"] / (data["balanceSheetHistory"]["balanceSheetHistory"]["balanceSheetStatements"][0]["totalLiab"]["raw"] + data["balanceSheetHistory"]["balanceSheetHistory"]["balanceSheetStatements"][0]["totalAssets"]["raw"])
        #interest_expense_current_year = data["incomeStatementHistory"]["incomeStatementHistory"]["incomeStatementHistory"][0]["interestExpense"]["raw"]
        #interest_expense_current_year = data["incomeStatementHistory"]["incomeStatementHistory"]["incomeStatementHistory"][1]["interestExpense"]["raw"]
        self.growth_rate = 0.12
        self.perpertuity_growth_rate = 0.07
        self.risk_free_rate = 0.0267
        self.market_risk_premium = 0.097 - self.risk_free_rate
        #weighted_average_cost_capital = debt_percentage * 0.06 + (1 - debt_percentage) * (risk_free_rate + equity_beta * market_risk_premium)
        self.discount_rate = self.risk_free_rate + self.equity_beta * self.market_risk_premium

        self.present_value = (self.fcf/(self.discount_rate - self.growth_rate))*(1-((1+self.growth_rate)/(1+self.discount_rate))**5) + (self.fcf/(self.discount_rate - self.perpertuity_growth_rate))*(1-((1+self.perpertuity_growth_rate)/(1+self.discount_rate))**20) 
        
        return self.present_value
    
    def get_intrinsic_pricing_discounted_cash_flow(self):
        self.shares_outstanding = self.data["defaultKeyStatistics"]["defaultKeyStatistics"]["sharesOutstanding"]["raw"]
        self.intrinsic_pricing = np.round(self.present_value / self.shares_outstanding, decimals=4)
        
        return self.intrinsic_pricing
    
    def get_intrinsic_discount(self):
        self.price = self.data["financialData"]["financialData"]["currentPrice"]["raw"]
        self.intrinsic_discount = f"{np.round((self.price / self.intrinsic_pricing)*100, decimals=2)}%"
        
        return self.intrinsic_discount
    
    def get_intrinsic_pricing_divividend_discount_model(self):
        try:
            self.dividend = GNTX.data["defaultKeyStatistics"]["defaultKeyStatistics"]["lastDividendValue"]["raw"]
        except:
            self.dividend = 0.00

        if self.dividend != 0.00:
                self.price_ddm = self.dividend / (self.discount_rate - self.perpetuity_growth_rate)
                
        return self.price_ddm
    
    def get_earnings_perfomance(self):
        self.income = {"year": "value"}
        start_year = 2018
        i = 0
        while i < 10:
            try:
                self.income[start_year-i] = self.data["incomeStatementHistory"]["incomeStatementHistory"]["incomeStatementHistory"][i]["netIncomeApplicableToCommonShares"]["raw"]
                i += 1
            except:
                break
                
    def search_web(self, search_term):
        url = f"https://api.cognitive.microsoft.com/bingcustomsearch/v7.0/search?q={self.name} {search_term}&customconfig={keys.custom_config_id}"
        r = requests.get(url, headers={'Ocp-Apim-Subscription-Key': keys.azure_subscription_key})
        #print(r.text.webPages)
        #print(r.text)
        
        return r.text




In [11]:
GNTX.data["defaultKeyStatistics"]

,defaultKeyStatistics
52WeekChange,"{'raw': -0.11515415, 'fmt': '-11.52%'}"
SandP52WeekChange,"{'raw': 0.00010061264, 'fmt': '0.01%'}"
annualHoldingsTurnover,{}
annualReportExpenseRatio,{}
beta,"{'raw': 1.023442, 'fmt': '1.02'}"
beta3Year,{}
bookValue,"{'raw': 7.179, 'fmt': '7.18'}"
category,None
dateShortInterest,"{'raw': 1550188800, 'fmt': '2019-02-15'}"
earningsQuarterlyGrowth,"{'raw': -0.185, 'fmt': '-18.50%'}"


In [27]:
GNTX = Analysis("GNTX")
GNTX.get_full_analysis()

In [28]:
GNTX.intrinsic_discount

'79.01%'

In [4]:
FB = Analysis("FB")
FB.get_full_analysis()

In [5]:
FB.intrinsic_discount

'221.21%'

In [12]:
for day in GNTX.prices.index:
    price_day = float(GNTX.prices["Time Series (Daily)"][day]["4. close"])
    for prior_day in GNTX.prices.index:
        price_prior_day = float(GNTX.prices["Time Series (Daily)"][prior_day]["4. close"])
        change = price_day - price_prior_day
        change_percent = change / price_prior_day
        print(change_percent)
"""        if  price_day > price_prior_day * 1.03:
            print("Big Move Up")
        elif  price_day < price_prior_day * 1.03:
            print("Big Move Down")
        else:
            pass"""

0.0
0.022175290390707397
0.03974221267454341
-0.028600100351229316
-0.05468750000000005
-0.07677634716261324
-0.0339321357285429
-0.03344982526210692
-0.05330073349633251
-0.0647342995169082
-0.07368421052631575
-0.08028503562945374
-0.10037174721189591
-0.0986964618249535
-0.10701107011070111
-0.10245711636532225
-0.11638521223185763
-0.12039981826442535
-0.11314704535043514
-0.11314704535043514
-0.12279111916628911
-0.14260407440212575
-0.15124945199473913
-0.1399378054198135
-0.1452538631346578
-0.1259593679458239
-0.1337807606263983
-0.13609995537706385
-0.13571428571428568
-0.12714156898106405
-0.14108251996450755
-0.13648528099910803
-0.14031971580817051
-0.14901098901098903
-0.11919927206551414
-0.10618651892890123
-0.07853403141361266
-0.0696780394041326
-0.07235265931959758
-0.08202939781887153
-0.062469733656174295
-0.048180924287118995
-0.03441396508728186
-0.03054581872809211
-0.03054581872809211
-0.03151575787893942
-0.025666834423754482
-0.0010319917440660255
-0.031031031

0.1446673706441393
0.16433941997851764
0.08780732563973909
0.058593749999999965
0.03385789222699098
0.0818363273453094
0.08237643534697946
0.06014669926650369
0.04734299516908215
0.03732057416267948
0.029928741092636532
0.007434944237918222
0.009310986964618215
0.0
0.0050996754751970064
-0.010497489730716587
-0.01499318491594738
-0.006871278057718671
-0.006871278057718671
-0.017671046669687385
-0.03985828166519038
-0.04953967558088553
-0.03687250111061758
-0.04282560706401761
-0.021218961625282116
-0.02997762863534683
-0.03257474341811693
-0.03214285714285709
-0.02254283137962128
-0.038154392191659246
-0.03300624442462096
-0.037300177619893425
-0.04703296703296705
-0.013648771610555083
0.0009233610341643386
0.031889576392194106
0.041806823642479625
0.03881169142309529
0.027975343764817442
0.0498789346246974
0.06588003933136675
0.0812967581047381
0.08562844266399604
0.08562844266399604
0.08454227113556785
0.09109209864116752
0.1186790505675955
0.08508508508508504
0.07968127490039849
0.0

-0.0004460303300625139
-0.004884547069271734
-0.014945054945054938
0.019563239308462224
0.03462603878116344
0.06663493574488331
0.07688611244593953
0.0737901293723047
0.06258890469416786
0.08523002421307514
0.1017699115044248
0.11770573566084785
0.12218327491236862
0.12218327491236862
0.12106053026513267
0.1278309008555611
0.15634674922600625
0.1216216216216216
0.11603585657370528
0.12953629032258066
0.10885700148441362
0.10231185440236114
0.11994002998500741
0.09745347698334957
0.09745347698334957
0.06612749762131306
0.03942486085343235
0.02002730996813843
0.015865820489573954
0.01540552786588128
0.00809716599190282
0.005383580080753746
0.0017881090746535158
-0.023529411764705847
-0.0066489361702127036
-0.003557136505113308
-0.007968127490039828
-0.0166739798157086
-0.020113686051596014
-0.02692140686061663
0.0071910112359550625
0.05807365439093486
0.06714285714285714
0.0696897374701671
0.07947976878612709
0.07896003851709199
0.1216216216216216
0.13353566009104706
0.1216216216216216
0

-0.0761398849048251
-0.08424747696358044
-0.08745080891998251
-0.09379070777247069
-0.062022471910112315
-0.014636449480642055
-0.006190476190476143
-0.0038186157517898947
0.00529865125240845
0.0048146364949447
0.044544544544544575
0.055639858371269675
0.044544544544544575
0.03573200992555844
0.04979879275653934
0.055639858371269675
0.02504911591355607
0.0185456320156175
0.02303921568627463
0.04402201100550288
0.03779214321233225
0.03779214321233225
0.026561731431382327
0.027572624322993712
0.0260570304818093
0.018048780487804925
0.036761053154495876
0.019540791402051887
0.019042968750000028
0.016066212268744006
0.019042968750000028
0.009675858732462472
0.008699855002416611
0.08935950413223143
0.11351636747624068
0.13265306122448972
0.0582037129954842
0.029785156249999972
0.005722460658083023
0.05239520958083836
0.052920619071392845
0.03129584352078243
0.018840579710144956
0.009090909090909153
0.0019002375296911708
-0.019981412639405192
-0.018156424581005613
-0.027214022140221394
-0.02

0.037190082644628045
0.06019007391763447
0.07841031149301811
0.007526342197691851
-0.019531250000000104
-0.04244158321411543
0.0019960079840318935
0.0024962556165749952
-0.01809290953545237
-0.029951690821256087
-0.039234449760765566
-0.04608076009501199
-0.066914498141264
-0.06517690875232784
-0.07380073800738014
-0.0690774223458508
-0.08352350524874495
-0.08768741481144948
-0.08016491067338526
-0.08016491067338526
-0.09016764839148174
-0.11071744906997344
-0.11968434896975014
-0.1079520213238562
-0.11346578366445918
-0.09345372460496616
-0.1015659955257272
-0.10397144132083899
-0.10357142857142859
-0.09467989179440944
-0.10913930789707191
-0.1043710972346121
-0.10834813499111906
-0.11736263736263744
-0.08644222020018208
-0.07294552169898438
-0.04426463588767269
-0.035079288803459896
-0.03785337805462399
-0.04788999525841638
-0.02760290556900728
-0.01278269419862348
0.001496259351620827
0.005508262393590357
0.005508262393590357
0.004502251125562774
0.010568696527428147
0.0361197110423

0.08119864668922183
0.18543388429752067
0.21172122492080242
0.23254564983888282
0.151530356246864
0.12060546874999994
0.09442060085836912
0.14520958083832336
0.1457813280079879
0.12224938875305624
0.10869565217391304
0.09808612440191392
0.09026128266033247
0.06644981412639404
0.06843575418994408
0.05857933579335792
0.06397774687065363
0.047466910086718356
0.04270786006360734
0.05130554283096661
0.05130554283096661
0.039873130946986815
0.016386182462356114
0.006137658921525672
0.0195468680586405
0.013245033112582814
0.03611738148984202
0.02684563758389252
0.024096385542168638
0.024553571428571463
0.034715960324616754
0.018189884649511986
0.023639607493309435
0.01909413854351686
0.00879120879120876
0.04413102820746127
0.05955678670360107
0.0923369823893383
0.10283517539644405
0.09966459032103489
0.08819345661450922
0.1113801452784504
0.12831858407079644
0.14463840399002487
0.14922383575363046
0.14922383575363046
0.1480740370185093
0.15500754906894806
0.1842105263157895
0.1486486486486486

0.014983083615273016
0.08212809917355371
0.10612460401267149
0.1251342642320085
0.05117912694430505
0.022949218749999944
-0.000953743443013809
0.04540918163672655
0.04593110334498243
0.02444987775061125
0.012077294685990338
0.0023923444976076897
-0.0047505938242280955
-0.026486988847583656
-0.024674115456238415
-0.03367158671586718
-0.02874362540565605
-0.043815609310817016
-0.048159927305770206
-0.04031149793861654
-0.04031149793861654
-0.05074762120525605
-0.07218777679362263
-0.081543182814555
-0.06930253220790769
-0.07505518763796906
-0.05417607223476295
-0.06263982102908286
-0.06514948683623387
-0.06473214285714284
-0.05545536519386837
-0.07054125998225377
-0.0655664585191794
-0.0697158081705151
-0.07912087912087916
-0.04686078252957239
-0.03277931671283476
-0.0028557829604951104
0.00672753483901973
0.003833253473885879
-0.006638217164532981
0.014527845036319648
0.029990167158308722
0.04488778054862835
0.049073610415623456
0.049073610415623456
0.04802401200600305
0.054353296426773

-0.09332732191163212
-0.10780834072759538
-0.10303300624442471
-0.10701598579040854
-0.11604395604395606
-0.08507734303912652
-0.0715604801477378
-0.04283674440742513
-0.03363767419509848
-0.0364159080019167
-0.046467520151730696
-0.026150121065375263
-0.011307767944936108
0.0029925187032418315
0.0070105157736605195
0.0070105157736605195
0.0060030015007504255
0.012078510317060816
0.037667698658410756
0.006506506506506457
0.001494023904382527
0.013608870967741915
-0.0049480455220188724
-0.01082144613871121
0.004997501249375205
-0.015181194906954076
-0.015181194906954076
-0.04329210275927688
-0.0672541743970315
-0.08466090122894855
-0.08839528558476878
-0.08880833710919804
-0.0953666216824112
-0.0978017048003589
-0.10102816271792586
-0.12374727668845316
-0.10859929078014181
-0.10582481102712313
-0.10978308986277116
-0.1175954365949978
-0.12068211630957593
-0.12679114198871044
-0.0961797752808989
-0.05051935788479699
-0.04238095238095241
-0.040095465393794744
-0.031310211946050194
-0.0317

0.0
-0.0009666505558240489
0.06869834710743812
0.09239704329461457
0.11117078410311494
0.03813346713497248
0.010253906250000042
-0.013352408202193496
0.03243512974051907
0.032950574138791815
0.011735941320293496
-0.00048309178743951745
-0.010047846889952024
-0.017102137767220874
-0.038568773234200666
-0.03677839851024205
-0.04566420664206635
-0.04079740380157622
-0.05568233683249652
-0.059972739663789194
-0.05222171323866226
-0.05222171323866226
-0.06252831898504753
-0.08370239149689979
-0.0929416922402454
-0.08085295424255887
-0.08653421633554073
-0.06591422121896151
-0.07427293064876958
-0.0767514502454261
-0.0763392857142856
-0.06717763751127134
-0.08207630878438323
-0.07716324710080287
-0.08126110124333918
-0.09054945054945049
-0.05868971792538667
-0.04478301015697132
-0.015230842455973358
-0.005766458433445336
-0.008624820316243398
-0.018966334755808372
0.001937046004842746
0.01720747295968542
0.03192019950124691
0.036054081121682645
0.036054081121682645
0.035017508754377334
0.041

'        if  price_day > price_prior_day * 1.03:\n            print("Big Move Up")\n        elif  price_day < price_prior_day * 1.03:\n            print("Big Move Down")\n        else:\n            pass'

24.434538868555055